# Einleitung

In den Straßen New York Citys herrscht ein hohes Verkehrsaufkommen, wodurch Verkehrsunfälle an der Tagesordnung sind. Durch das NYC Open Data Project (https://opendata.cityofnewyork.us/) werden die erfassten Verkehrsunfälle für die Allgemeinheit zugänglich gemacht.

Thema dieses Berichtes und Thema unseres Projektes ist es, die öffentlich zugänglichen Unfalldaten zu nutzen, um darauf die im Abschnitt `Analyseziele` beschriebenen Analysen durchzuführen und die Ergebnisse zu visualisieren.

## Aufbau der Data Pipeline

Für das Projekt soll eine Data-Pipeline aufgebaut werden, welche die Daten in die Datenbank lädt. 
Von dort aus sollen die Daten von einem Python-Script zur Analyse angefragt werden.

Beteiligt an der Data-Pipeline ist eine Kafka-Instanz, wobei ein Python-Script einen Producer dafür darstellt und ein weiteres Python-Script einen Consumer. Das Producer-Script lädt die Daten zeilenweise aus der Datenquelle, einer CSV-Datei, in Kafka ein, während der Consumer die Daten aus Kafka ausliest und in die MongoDB schreibt. Das Analyse-Script bezieht die Daten wiederum aus der MongoDB.

### Installation der benötigten Docker-Container

Das Script `start_docker.sh` startet den MongoDB Docker Container, beziehungsweise erstellt bei Bedarf einen neuen Docker Container. Dabei wird, falls noch kein Image für die DockerDB vorhanden ist, das neueste Heruntergeladen. 

Die weiteren benötigten Docker Container sind in der Docker-Compose-Konfiguration (`docker-compose.yml`) vorhanden.
Diese können mit `docker-compose up` erstellt oder heruntergeladen werden. 
Mit `docker-compose down` können die gestarteten Container gestoppt werden, wobei auf die Vollendigung des Befehls gewartet werden sollte.

Auf manchen Systemen befindet sich die Docker Engine nach stoppen der Docker Container mit `docker-compose down` in einem nicht-revertierbarem Fehlerzustand, wobei ein _Neustart des Computers_ unausweichlich ist. Weitere Informationen zu dem Fehler, wobei jedoch _keine der angegebenen Workarounds genutzt werden sollte_, findet sich [auf GitHub](https://github.com/docker/for-linux/issues/162). 

## Datenquelle

Als Datenquellen dienen die vom NYC Open Data Project bereitgestellten Unfalldaten des NYPD. Die detaillierten Unfalldaten befinden sich in 3 unterschiedlichen Datenquellen die unter Anderem als CSV-Datei verfügbar sind: [Motor Vehicle Collisions](https://data.cityofnewyork.us/browse?Data-Collection_Data-Collection=Motor+Vehicle+Collisions&q=crashes).

Enthalten in den Daten sind die am Unfall beteiligten Verkehrsteilnehmer, die Verletzten und Toten, sowie die genaue Position des Unfalls. Darüberhinaus sind noch weitere Daten in den Quellen enthalten, auf die teilweise im Analyseabschnitt genauer eingegangen wird.

## Analyseziele

Ziel der Analyse der Daten ist herauszufinden, welche saisonalen und lokalen Zusammenhänge festzustellen sind, wodurch sich besondere Hotspots im Stadtraum New Yorks lokalisieren lassen. Des weiteren soll ermittelt werden, welche Verkehrsteilnehmer besonders oft getötet oder verletzt werden, sowie an welchen Stellen sich gerade schwerwiegende Unfälle häufen. Anhand einer animierten Karte kann dadurch das Unfallgeschehen über die Zeit betrachtet werden.

## Installation der Python Packages

Damit nicht alles vollständig von uns entwickelt werden muss, wurden verschiedene Bibliotheken verwendet. Auf diese Weise kann sich die Umsetzung auf das Wesentliche beziehen.

Apache Kafka (https://kafka.apache.org/) soll dazu genutzt werden, mit Hilfe eines Producers und eines Consumers die Datensätze zeilenweise einzulesen. Dies wird durch die kafka-python Bibliothek (https://github.com/dpkp/kafka-python) stark vereinfacht.

Während der Kafka Consumer die Datensätze registiert und ausliest müssen diese ein die MongoDB eingetragen werden. Um dieses Verfahren zu vereinfachen, wird die pymongo Bibliothek (https://pypi.org/project/pymongo/) genutzt.

Es folgt die Installation der Bibliotheken über `pip install`:

In [ ]:
pip install kafka-python pymongo Cartopy folium rasterio earthpy

### Importieren benötigter Module

Damit die verschiedenen Funktionen und Datentypen der Bibliotheken genutzt werden können, müssen die entsprechenden Module importiert werden. Im folgenden Abschnitt werden alle notwendigen Module, die im Laufe des Projektes genutzt werden importiert.

Zusätzlich hat sich während der Entwicklung gezeigt, dass je nach Betriebssystem Unterschiede bezüglich der Paralellisierung von Prozessen entstehen. Aus diesem Grund wird hier eine Variable `windows` deklariert, mit der im Laufe des zwischen Windows und anderen Betriebssystemen unterschieden werden kann.

Zuletzt wird hierbei die Breite des Jupyter Notebooks auf 100% gestellt, um die Lesbarkeit zu verbessern.

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
from pymongo import MongoClient
import datetime as dt
import requests
import os
from pathlib import Path
from multiprocessing import Process
from time import sleep
from data.schemas import schemas
import platform
windows = True if 'Windows' in platform.system() else False 

from IPython.core.display import display, HTML
display(HTML(
    "<style>.container { width:100% !important; }</style>"
))

DEBUG = True

### Herunterladen der Datensätze
Zum Herunterladen der Datensätze werden die Daten von der Datenquelle mithilfe des nachfolgenden Python-Scripts heruntergeladen. Resultat sind drei `.csv`-Dateien, welche die Unfalldaten zeilenweise enthalten.
Die Dateien werden im Ordner `data` gespeichert, damit sie nur einmalig heruntergeladen werden müssen. Der Ordner befindet sich im gleichen Verzeichnis wie dieses Jupyter Notebook.

In [ ]:
if not os.path.exists('data/'):
    os.mkdir('data')

for file_name, download_url in [
    ('crashes.csv', 'https://data.cityofnewyork.us/api/'
     'views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'),
    ('vehicles.csv', 'https://data.cityofnewyork.us/api/'
     'views/bm4k-52h4/rows.csv?accessType=DOWNLOAD'),
    ('persons.csv', 'https://data.cityofnewyork.us/api/'
     'views/f55k-p6yu/rows.csv?accessType=DOWNLOAD'),
]:
    if not os.path.isfile(fp:= (Path('data') / file_name)):
        with open(fp, 'wb') as crash_file:
            crash_file.write(requests.get(download_url).content)

# Einlesen der Daten und senden über den Producer

## Produzieren der CSV-Daten in Kafka

In [ ]:
from tools import send_to_kafka
                
producers = {
    'crashes': Process(
        target=send_to_kafka, args=[
            'crashes', 5000 if DEBUG else None
    ]),
    'vehicles': Process(
        target=send_to_kafka, args=[
            'vehicles', 5000 if DEBUG else None
    ]),
    'persons': Process(
        target=send_to_kafka, args=[
            'persons', 5000 if DEBUG else None
    ]),
}

## Konsumieren der Daten und Import in die Datenbank

Zunächst werden die Daten aus den Topics `nyc_persons` und `nyc_vehicles` verarbeitet.

In [ ]:
from tools import process_topic

consumers = {
    'vehicles': Process(
        target=process_topic, args=(
        'vehicles', schemas['vehicles']
    )),
    'persons': Process(
        target=process_topic, args=(
        'persons', schemas['persons']
    )),
    'crashes': Process(
        target=process_topic, args=(
        'crashes', schemas['crashes']
    )),
}

## Starten der Prozesse
Nun werden die Prozesse zum Produzieren der Daten aus `vehicles.csv`, `persons.csv`, und `crashes.csv` und die Prozesse zum Konsumieren der Topics `nyc_vehicles` und `nyc_persons` gestartet.

In [ ]:
if windows:
    limit = send_to_kafka('vehicles', 5000 if DEBUG else None)
    process_topic('vehicles', schemas['vehicles'], limit)
    limit = send_to_kafka('persons', 5000 if DEBUG else None)
    process_topic('persons', schemas['persons'], limit)
    limit = send_to_kafka('crashes', 5000 if DEBUG else None)
    process_topic('crashes', schemas['crashes'], limit)
else:
    for topic in 'persons vehicles crashes'.split(' '):
        producers[topic].start()
    sleep(2)
    for topic in 'persons vehicles'.split(' '):
        consumers[topic].start()

## Starten des Join-Prozesses (nicht auf Windows)
Im letzten Schritt muss der Prozess gestartet werden, der das joinen der beiden Tabellen `vehicles` und `persons` in die Daten des Topics `nyc_crashes` durchführt.

Wichtig ist hierbei, dass die parallelisierten Prozesse zum Produzieren aller Daten und zum Einlesen der `nyc_vehicles` und `nyc_persons` Topics beendet sind.

In [ ]:
if not windows:
    assert False, 'breakpoint'

In [ ]:
if not windows:
    consumers['crashes'].start()

# Datenanalyse

# Ergebnis